In [1]:
import pickle 
import numpy as np
from numpy.linalg import norm
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D 
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2 

In [2]:
feature_list = np.array(pickle.load(open('Embeddings.pkl','rb')))
feature_list.shape

(44441, 2048)

In [3]:
filenames = pickle.load(open('Filenames.pkl','rb'))

In [4]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False
model = tensorflow.keras.Sequential([model,GlobalMaxPooling2D()])

In [5]:
img = image.load_img('Data/Sample/T-Shirt.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result/norm(result)

1/1 [==============================] - 1s 1s/step


##### Now we need to calculate the distance between normalized_result and feature_list by using nearest neighbours
      algorithm is brute becasue dataset is small 

##### Euclidean Distance

In [6]:
neighbours = NearestNeighbors(n_neighbors=5,algorithm='brute',metric='chebyshev')
neighbours.fit(feature_list)        #Our input data is feature_list 

NearestNeighbors(algorithm='brute', metric='chebyshev')

'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis', 'hamming', 'canberra', 'braycurtis', 'haversine', 'cosine', 'correlation', 'dice', 'kulsinski', 'rogerstanimoto', 'russellrao', 'sokalmichener', 'sokalsneath', 'yule'

##### Calculating nearest_neighbours to normalized_result
      The function returns us the distances and the indices of the nearest neighbours

In [7]:
distances,indices = neighbours.kneighbors([normalized_result])
print(indices)

[[ 3152 25048 27557   441 16619]]


In [8]:
for file in indices[0]:
    print(filenames[file])

E:\Computer Science\Machine Learning\Projects\Fashion Recommendation System\Data\images\13713.jpg
E:\Computer Science\Machine Learning\Projects\Fashion Recommendation System\Data\images\40023.jpg
E:\Computer Science\Machine Learning\Projects\Fashion Recommendation System\Data\images\43506.jpg
E:\Computer Science\Machine Learning\Projects\Fashion Recommendation System\Data\images\10486.jpg
E:\Computer Science\Machine Learning\Projects\Fashion Recommendation System\Data\images\29288.jpg


In [9]:
for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('OUTPUT',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)